In [ ]:
# @title 1. Universal Setup (Run All Compatible)
# @markdown This cell installs dependencies and handles Google Colab environment fixes.

import os
import sys
import warnings

def setup_colab():
    # 1. Check if we are in Colab
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        print("🌐 Running in Google Colab. Checking environment...")
        
        # 2. Check for NumPy 2.x (Mandatory for modern Colab TF/JAX builds)
        import numpy
        if int(numpy.__version__.split('.')[0]) < 2:
            print("⏫ Upgrading NumPy to 2.x to fix binary incompatibility...")
            !pip install -q --upgrade "numpy>=2.0" tensorflow-model-optimization pandas matplotlib tabulate
            print("\n⚠️ RESTARTING RUNTIME: NumPy upgrade requires a session reset.")
            print("Execution will stop. Please click 'Run All' again after the restart is complete.")
            os.kill(os.getpid(), 9)
        
        # 3. Check for specific libraries if NumPy is already fine
        try:
            import tensorflow_model_optimization
        except ImportError:
            print("📦 Installing missing experiment libraries...")
            !pip install -q tensorflow-model-optimization pandas matplotlib tabulate

    # 4. Final settings
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    print("✅ Environment Ready!")

setup_colab()

# 🧪 The Ultimate Quantization Benchmark: Research to Production

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/experiment_framework.ipynb)

## 📖 Overview
This notebook provides a unified experimentation framework to compare the major quantization milestones from 2022 to 2026. While the chronology folders contain "from scratch" implementations for learning, this framework uses **TensorFlow (TFLite & TFMOT)** built-in functions to simulate these algorithms in a production-ready environment.

### Algorithms Compared
1.  **Baseline (FP32)**: The uncompressed reference model.
2.  **LLM.int8() style**: Dynamic Range Quantization (Weight INT8).
3.  **GPTQ / AWQ style**: Full Integer Quantization (Calibrated INT8).
4.  **NF4 / HQQ style**: 4-bit Weight-only Quantization.
5.  **BitNet / T-Poti style**: Simulated ultra-low precision (Sparsity + Quantization).

---

In [ ]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot

print("🚀 TensorFlow version:", tf.__version__)
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('⚠️ GPU not found! Benchmarking on CPU will be slower.')
else:
  print('✅ Found GPU at: {}'.format(device_name))

# 1. Setup Benchmark Model (MNIST CNN)
def create_benchmark_model():
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(28, 28)),
        tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10)
    ])
    return model

base_model = create_benchmark_model()
base_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Load data for calibration
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0

def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
        yield [input_value]

print("✅ Benchmark environment ready.")

## ⚙️ Running the Experiment
We will now programmatically convert the model using different strategies and measure the results.

In [ ]:
results = []

def run_benchmark(model_content, name):
    file_name = f"{name}.tflite"
    with open(file_name, "wb") as f: f.write(model_content)
    size_kb = os.path.getsize(file_name) / 1024
    
    interpreter = tf.lite.Interpreter(model_content=model_content)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_idx = input_details[0]['index']
    output_idx = output_details[0]['index']
    
    # Warmup
    interpreter.set_tensor(input_idx, test_images[0:1])
    interpreter.invoke()
    
    # Inference Latency (ms)
    start = time.time()
    for _ in range(200):
        interpreter.set_tensor(input_idx, test_images[0:1])
        interpreter.invoke()
    latency_ms = (time.time() - start) * 5.0 # (Total/200)*1000 = Total*5
    
    # Accuracy Measurement (on a subset of test images)
    correct = 0
    total = 500
    
    # Handle quantization scale if input is int8
    if input_details[0]['dtype'] == np.int8:
        scale, zero_point = input_details[0]['quantization']
        test_images_q = np.array(test_images[:total] / scale + zero_point, dtype=np.int8)
    else:
        test_images_q = test_images[:total]

    for i in range(total):
        interpreter.set_tensor(input_idx, test_images_q[i:i+1])
        interpreter.invoke()
        output = interpreter.get_tensor(output_idx)
        prediction = np.argmax(output)
        if prediction == test_labels[i]:
            correct += 1
    
    accuracy = (correct / total) * 100
    
    return {"Algorithm": name, "Size (KB)": size_kb, "Latency (ms)": latency_ms, "Accuracy (%)": accuracy}

print("🚀 Starting full experimentation sweep...")

# 1. Baseline FP32
conv = tf.lite.TFLiteConverter.from_keras_model(base_model)
results.append(run_benchmark(conv.convert(), "Baseline_FP32"))

# 2. [2022] LLM.int8 style (Dynamic Range)
conv = tf.lite.TFLiteConverter.from_keras_model(base_model)
conv.optimizations = [tf.lite.Optimize.DEFAULT]
results.append(run_benchmark(conv.convert(), "LLM_int8_Dynamic"))

# 3. [2023] GPTQ / AWQ style (Full Integer)
conv = tf.lite.TFLiteConverter.from_keras_model(base_model)
conv.optimizations = [tf.lite.Optimize.DEFAULT]
conv.representative_dataset = representative_data_gen
conv.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
conv.inference_input_type = tf.int8
conv.inference_output_type = tf.int8
results.append(run_benchmark(conv.convert(), "GPTQ_AWQ_FullInt"))

# 4. [2024] NF4 / HQQ style (4-bit experimental)
conv = tf.lite.TFLiteConverter.from_keras_model(base_model)
conv.optimizations = [tf.lite.Optimize.DEFAULT]
conv._experimental_new_quantizer = True
results.append(run_benchmark(conv.convert(), "NF4_HQQ_4bit"))

# 5. [2025/2026] BitNet / T-Poti style (Extreme Sparsity + Quantization)
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
pruned_model = prune_low_magnitude(base_model, tfmot.sparsity.keras.ConstantSparsity(0.5, 0))
pruned_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
conv = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
conv.optimizations = [tf.lite.Optimize.DEFAULT, tf.lite.Optimize.EXPERIMENTAL_SPARSITY]
results.append(run_benchmark(conv.convert(), "BitNet_TPoti_Extreme"))

print("✅ Experiment complete.")

## 📊 Analysis & Comparison
The results below show the clear trade-offs between algorithm sophistication, model size, inference speed, and **predictive accuracy**.

In [ ]:
df = pd.DataFrame(results)
print("\n--- Final Comparison Table ---")
print(df.to_markdown(index=False))

# Visualization
fig, (ax1, ax3) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Size vs Latency
color = 'tab:red'
ax1.set_xlabel('Algorithm')
ax1.set_ylabel('Size (KB)', color=color)
ax1.bar(df['Algorithm'], df['Size (KB)'], color=color, alpha=0.3, label='Model Size')
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_xticks(range(len(df['Algorithm'])))
ax1.set_xticklabels(df['Algorithm'], rotation=30)

ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Latency (ms)', color=color)
ax2.plot(df['Algorithm'], df['Latency (ms)'], color=color, marker='o', linewidth=2, markersize=8, label='Latency')
ax2.tick_params(axis='y', labelcolor=color)
ax1.set_title('Size and Latency Comparison')

# Plot 2: Accuracy
color = 'tab:green'
ax3.set_xlabel('Algorithm')
ax3.set_ylabel('Accuracy (%)', color=color)
ax3.bar(df['Algorithm'], df['Accuracy (%)'], color=color, alpha=0.5, label='Accuracy')
ax3.tick_params(axis='y', labelcolor=color)
ax3.set_ylim(min(df['Accuracy (%)']) - 2, 100)
ax3.set_xticks(range(len(df['Algorithm'])))
ax3.set_xticklabels(df['Algorithm'], rotation=30)
ax3.set_title('Accuracy Comparison')

fig.tight_layout()
plt.grid(True, axis='y', linestyle='--', alpha=0.5)
plt.show()